In [6]:
!pip install trafilatura feedparser apscheduler pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.8 MB/s eta 0:00:00


In [7]:
import time
import sqlite3
import feedparser
import trafilatura
from datetime import datetime
from apscheduler.schedulers.background import BackgroundScheduler
from pydantic import BaseModel

In [12]:
# ==========================================
# 1. 환경 설정 (Configuration)
# ==========================================
DB_FILE = "finiary.db"           # 데이터가 저장될 DB 파일명
UPDATE_INTERVAL_MINUTES = 1     # 뉴스 확인 주기 (분)

In [13]:
# ==========================================
# 2. 데이터 모델 (Data Structure)
# ==========================================
class StockNews(BaseModel):
    title: str
    link: str
    source: str
    pub_date: str
    content: str  # 본문 전체

In [14]:
# ==========================================
# 3. Finiary 수집 엔진 (Core Logic)
# ==========================================
class FiniaryEngine:
    def __init__(self):
        # 주식/시황/증권 전문 RSS 소스
        self.rss_sources = {
            "한국경제(증권)": "https://rss.hankyung.com/feed/market",
            "매일경제(증권)": "https://www.mk.co.kr/rss/50300009/",
            "서울경제(증권)": "https://www.sedaily.com/RSS/Stock",
            "이데일리(증권)": "https://rss.edaily.co.kr/stock_news.xml",
            "아시아경제(증권)": "https://rss.asiae.co.kr/stock.xml"
        }
        self.init_database()

    def init_database(self):
        """DB 테이블이 없으면 생성"""
        try:
            conn = sqlite3.connect(DB_FILE)
            cursor = conn.cursor()
            # link 컬럼에 UNIQUE 제약조건을 걸어 중복 저장을 원천 차단
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS news (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    title TEXT,
                    link TEXT UNIQUE,
                    source TEXT,
                    pub_date TEXT,
                    content TEXT,
                    ai_summary TEXT,      -- 추후 AI 요약 저장용
                    ai_sentiment TEXT,    -- 추후 AI 호재/악재 저장용
                    collected_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            conn.commit()
            conn.close()
            print(f"✅ 데이터베이스 연결 완료: {DB_FILE}")
        except Exception as e:
            print(f"❌ DB 초기화 실패: {e}")

    def is_duplicate(self, link):
        """이미 수집한 뉴스인지 DB에서 확인"""
        conn = sqlite3.connect(DB_FILE)
        cursor = conn.cursor()
        cursor.execute('SELECT 1 FROM news WHERE link = ?', (link,))
        exists = cursor.fetchone()
        conn.close()
        return exists is not None

    def crawl_full_body(self, url):
        """링크 타고 들어가서 본문 전체 추출 (Trafilatura 사용)"""
        try:
            downloaded = trafilatura.fetch_url(url)
            if downloaded:
                # 표(Table) 포함, 댓글 제외하고 본문 추출
                text = trafilatura.extract(downloaded, include_tables=True, include_comments=False)
                return text
            return None
        except Exception:
            return None

    def save_to_db(self, news: StockNews):
        """DB에 뉴스 저장"""
        try:
            conn = sqlite3.connect(DB_FILE)
            cursor = conn.cursor()
            cursor.execute('''
                INSERT OR IGNORE INTO news (title, link, source, pub_date, content)
                VALUES (?, ?, ?, ?, ?)
            ''', (news.title, news.link, news.source, news.pub_date, news.content))
            conn.commit()
            conn.close()
            return True
        except Exception as e:
            print(f"❌ 저장 중 오류 발생: {e}")
            return False

    def job(self):
        """스케줄러가 주기적으로 실행할 작업"""
        print(f"\n⏰ [자동수집 시작] {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        new_items_count = 0

        for source_name, rss_url in self.rss_sources.items():
            # RSS 피드 로딩 (가벼운 작업)
            feed = feedparser.parse(rss_url)
            
            # 최신순으로 들어오므로, 효율을 위해 상위 10개만 검사해도 충분
            for entry in feed.entries[:10]:
                link = entry.link
                title = entry.title

                # 1. 중복 검사 (DB에 있으면 건너뜀)
                if self.is_duplicate(link):
                    continue

                print(f"   🆕 신규 발견: {title[:20]}...", end="")

                # 2. 본문 크롤링
                content = self.crawl_full_body(link)

                # 본문이 없거나 너무 짧으면(영상/광고) 저장 안 함
                if not content or len(content) < 200:
                    print(" 🚫 [Pass: 본문 짧음]")
                    continue

                # 3. 객체 생성 및 저장
                news_item = StockNews(
                    title=title,
                    link=link,
                    source=source_name,
                    pub_date=entry.get('published', datetime.now().strftime('%Y-%m-%d')),
                    content=content
                )
                
                self.save_to_db(news_item)
                new_items_count += 1
                print(" ✅ 저장 완료")

                # 서버 부하 방지 (0.5초 대기)
                time.sleep(0.5)

        if new_items_count == 0:
            print("💤 새로운 뉴스가 없습니다.")
        else:
            print(f"🎉 업데이트 완료! 총 {new_items_count}개의 새 기사를 저장했습니다.")

In [15]:
# ==========================================
# 4. 실행 (Execution)
# ==========================================
if __name__ == "__main__":
    # 1. 엔진 초기화
    engine = FiniaryEngine()
    
    # 2. 스케줄러 설정 (백그라운드 실행)
    scheduler = BackgroundScheduler()
    scheduler.add_job(engine.job, 'interval', minutes=UPDATE_INTERVAL_MINUTES)
    scheduler.start()

    # 3. 최초 1회 즉시 실행 (기다리지 않고 바로 확인)
    engine.job()

    print("\n" + "="*60)
    print(f"🚀 Finiary 수집기 가동 중 (주기: {UPDATE_INTERVAL_MINUTES}분)")
    print(f"📂 데이터베이스: {DB_FILE}")
    print("종료하려면 Ctrl+C를 누르세요.")
    print("="*60)

    # 4. 프로그램이 꺼지지 않게 유지
    try:
        while True:
            time.sleep(1)
    except (KeyboardInterrupt, SystemExit):
        print("🛑 수집기를 종료합니다.")
        scheduler.shutdown()

✅ 데이터베이스 연결 완료: finiary.db

⏰ [자동수집 시작] 2026-01-27 17:40:36
💤 새로운 뉴스가 없습니다.

🚀 Finiary 수집기 가동 중 (주기: 1분)
📂 데이터베이스: finiary.db
종료하려면 Ctrl+C를 누르세요.

⏰ [자동수집 시작] 2026-01-27 17:41:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:42:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:43:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:44:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:45:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:46:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:47:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:48:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:49:36
   🆕 신규 발견: "입지만 좋다면 작아도 돼" … 한강... ✅ 저장 완료
   🆕 신규 발견: 공급대책 발표도 안 됐는데 … 과천·... ✅ 저장 완료
🎉 업데이트 완료! 총 2개의 새 기사를 저장했습니다.

⏰ [자동수집 시작] 2026-01-27 17:50:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:51:36
💤 새로운 뉴스가 없습니다.

⏰ [자동수집 시작] 2026-01-27 17:52:36
💤 새로운 뉴스가 없습니다.
🛑 수집기를 종료합니다.
